In [1]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

Using TensorFlow backend.


In [17]:
!git clone https://github.com/rahulvigneswaran/Intrusion-Detection-Systems

Cloning into 'Intrusion-Detection-Systems'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 151 (delta 36), reused 39 (delta 24), pack-reused 93
Receiving objects: 100% (151/151), 13.30 MiB | 14.17 MiB/s, done.
Resolving deltas: 100% (89/89), done.
Checking out files: 100% (68/68), done.


In [0]:
traindata = pd.read_csv('Intrusion-Detection-Systems/kddtrain.csv', header=None)
testdata = pd.read_csv('Intrusion-Detection-Systems/kddtest.csv', header=None)

In [26]:
traindata

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,0,0,1,1,1,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.0,0.00,1.0,0.0,0.00,9,9,1.0,0.0,0.11,0.00,0.00,0.00,0.00,0.0
1,0,0,1,1,1,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.0,0.00,1.0,0.0,0.00,19,19,1.0,0.0,0.05,0.00,0.00,0.00,0.00,0.0
2,0,0,1,1,1,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.0,0.00,1.0,0.0,0.00,29,29,1.0,0.0,0.03,0.00,0.00,0.00,0.00,0.0
3,0,0,1,1,1,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.0,0.00,1.0,0.0,0.00,39,39,1.0,0.0,0.03,0.00,0.00,0.00,0.00,0.0
4,0,0,1,1,1,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.0,0.00,1.0,0.0,0.00,49,49,1.0,0.0,0.02,0.00,0.00,0.00,0.00,0.0
5,0,0,1,1,1,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.0,0.00,1.0,0.0,0.00,59,59,1.0,0.0,0.02,0.00,0.00,0.00,0.00,0.0
6,0,0,1,1,1,212,1940,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.00,0.00,0.0,0.00,1.0,0.0,1.00,1,69,1.0,0.0,1.00,0.04,0.00,0.00,0.00,0.0
7,0,0,1,1,1,159,4087,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.00,0.00,0.0,0.00,1.0,0.0,0.00,11,79,1.0,0.0,0.09,0.04,0.00,0.00,0.00,0.0
8,0,0,1,1,1,210,151,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.0,0.00,1.0,0.0,0.00,8,89,1.0,0.0,0.12,0.04,0.00,0.00,0.00,0.0
9,0,0,1,1,1,212,786,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.0,0.00,1.0,0.0,0.00,8,99,1.0,0.0,0.12,0.05,0.00,0.00,0.00,0.0


In [14]:
traindata.shape

(25192, 43)

In [0]:
X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,0]
C = testdata.iloc[:,0]
T = testdata.iloc[:,1:42]

In [0]:
scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
#print(trainX[0:5,:])

scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])


y_train = np.array(Y)
y_test = np.array(C)

In [29]:

# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))


print(X_train.shape)


batch_size = 32

(494021, 1, 41)


In [35]:
# 1. define the network
model = Sequential()
model.add(GRU(4,input_dim=41))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))
print(model.get_config())


# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/lstm1layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('training_set_iranalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1000, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
model.save("kddresults/lstm1layer/fullmodel/lstm1layer_model.hdf5")

loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = model.predict_classes(X_test)
np.savetxt('kddresults/lstm1layer/lstm1predicted.txt', y_pred, fmt='%01d')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(4, input_shape=(None, 41))`
  


{'name': 'sequential_6', 'layers': [{'class_name': 'GRU', 'config': {'name': 'gru_2', 'trainable': True, 'batch_input_shape': (None, None, 41), 'dtype': 'float32', 'return_sequences': False, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'units': 4, 'activation': 'tanh', 'recurrent_activation': 'hard_sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.0, 'recurrent_dropout': 0.0, 'implementation': 1, 'reset_after': False}}, {'class_name': 'Dropout', 'config': {'name': 'dropout_6

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  del sys.path[0]


Train on 494021 samples, validate on 311029 samples
Epoch 1/1000
494021/494021 [==============================] - 94s 190us/step - loss: 0.0566 - acc: 0.9808 - val_loss: 0.5203 - val_acc: 0.8620

Epoch 00001: val_acc improved from -inf to 0.86205, saving model to kddresults/lstm1layer/checkpoint-01.hdf5
Epoch 2/1000
494021/494021 [==============================] - 92s 186us/step - loss: 0.0310 - acc: 0.9871 - val_loss: 0.8635 - val_acc: 0.8136

Epoch 00002: val_acc did not improve from 0.86205
Epoch 3/1000
494021/494021 [==============================] - 91s 185us/step - loss: 0.0270 - acc: 0.9889 - val_loss: 1.0822 - val_acc: 0.8113

Epoch 00003: val_acc did not improve from 0.86205
Epoch 4/1000
494021/494021 [==============================] - 92s 186us/step - loss: 0.0240 - acc: 0.9904 - val_loss: 1.1261 - val_acc: 0.8118

Epoch 00004: val_acc did not improve from 0.86205
Epoch 5/1000
494021/494021 [==============================] - 93s 188us/step - loss: 0.0218 - acc: 0.9915 - val_l

In [0]:
!mkdir kddresults/lstm1layer/fullmodel

In [36]:
!ls kddresults/lstm1layer

checkpoint-01.hdf5   checkpoint-16.hdf5  checkpoint-44.hdf5  checkpoint-67.hdf5
checkpoint-08.hdf5   checkpoint-22.hdf5  checkpoint-46.hdf5  checkpoint-70.hdf5
checkpoint-10.hdf5   checkpoint-25.hdf5  checkpoint-53.hdf5  checkpoint-84.hdf5
checkpoint-112.hdf5  checkpoint-26.hdf5  checkpoint-54.hdf5  checkpoint-87.hdf5
checkpoint-11.hdf5   checkpoint-32.hdf5  checkpoint-57.hdf5  checkpoint-99.hdf5
checkpoint-12.hdf5   checkpoint-35.hdf5  checkpoint-63.hdf5  fullmodel
checkpoint-14.hdf5   checkpoint-36.hdf5  checkpoint-65.hdf5


In [38]:
testdata = pd.read_csv('Intrusion-Detection-Systems/kddtest.csv', header=None)



C = testdata.iloc[:,0]
T = testdata.iloc[:,1:42]


scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])



y_test = np.array(C)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))


batch_size = 32

# 1. define the network
model = Sequential()
model.add(GRU(4,input_dim=41))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))


# try using different optimizers and different optimizer configs
model.load_weights("kddresults/lstm1layer/checkpoint-99.hdf5")

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
y_train1 = y_test

y_pred = model.predict_classes(X_train)
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="binary")
precision = precision_score(y_train1, y_pred , average="binary")
f1 = f1_score(y_train1, y_pred, average="binary")

print("confusion matrix")
print("----------------------------------------------")
print("accuracy")
print("%.3f" %accuracy)
print("racall")
print("%.3f" %recall)
print("precision")
print("%.3f" %precision)
print("f1score")
print("%.3f" %f1)
cm = metrics.confusion_matrix(y_train1, y_pred)
print("==============================================")
print("==============================================")

print(cm)
tp = cm[0][0]
fp = cm[0][1]
tn = cm[1][1]
fn = cm[1][0]
print("tp")
print(tp)
print("fp")
print(fp)
print("tn")
print(tn)
print("fn")
print(fn)
print("tpr")
tpr = float(tp)/(tp+fn)
print("fpr")
fpr = float(fp)/(fp+tn)
print("LSTM acc")
print(tpr)
print(fpr)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
loss, accuracy = model.evaluate(X_train, y_train1)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))

t_probs = model.predict_proba(X_train)
print(t_probs)
np.savetxt('prob.txt', t_probs)
print(t_probs.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(4, input_shape=(None, 41))`


confusion matrix
----------------------------------------------
accuracy
0.925
racall
0.908
precision
0.999
f1score
0.951
[[ 60268    325]
 [ 23109 227327]]
tp
60268
fp
325
tn
227327
fn
23109
tpr
fpr
LSTM acc
0.7228372332897561
0.001427617591762866
311029/311029 [==============================] - 17s 55us/step

Loss: 1.04, Accuracy: 92.47%
[[0.000e+00]
 [0.000e+00]
 [0.000e+00]
 ...
 [0.000e+00]
 [0.000e+00]
 [7.449e-33]]
(311029, 1)
